In [ ]:
For agents like "Itinerary Planner", where no tools are needed and you just want a clear response, 
we do not require to use agents at all.
We can achieve the same thing with LLMChain. In this the agents were not explicitly defined as standalone objects. 
Instead, LLMChain instances are used to perform the roles of agents.

Langchain has two levels of abstraction of AI_driven workflows:
1. LLMChain (these are not full "agents" - they are deterministic chains of prompt + LLM). 
Suitable for sequential tasks. Not decision-making, tool selection, or memory
2. LangChain Agents (using this, we will define true agents. initialize_agents, AgentExecutor+tools, Tool use + memory + reasoning)

1.Setup & Environment

In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from Langchain.tools import tool
from langhchain_community.tools.tavily_search import TavilySearchResults

In [ ]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
tavily_api_key = os.getenv("TAVILY_API_KEY")

if not openain_api_key or not tavily_api_key:
    raise ValueError("Missing OpenAI or Tavily API keys")

In [ ]:
llm = ChatOpenAI(model="gpt-4o", temperature=0.7, openai_api_key=openai_api_key)

2. Travily search tool

In [ ]:
@tool
def search_internet(query: str) -> str:
    """Search the internet and return formatted results."""
    search = TavilySearchResults(api_key=tavily_api_key, max_results=5)
    results = search.invoke(query)
    return "\n\n".join(
        f"Source: {r.get('url', 'No URL')}\n"
        f"Title: {r.get('title', 'No Title')}\n"
        f"Content: {r.get('content', 'No Content')}"
        for r in results
    )

3. Prompt Template for each role

destination_prompt = PromptTemplate.from_template("""
Research comprehensive information about {destination}:
- Best seasons, safety, etiquette, travel options, costs, language, etc.
Travel Dates: {travel_dates}, Budget: {budget}.
""")

attractions_prompt = PromptTemplate.from_template("""
Identify the top attractions in {destination} for {preferences} travelers on a {budget} budget,
Trip Duration: {duration_days} days.
""")

itinerary_prompt = PromptTemplate.from_template("""
Created a detailed {duration_days}-day itinerary for {destination} starting {travel_dates}.
Preferences: {preferences}, Budget: {budget}.
""")

guide_prompt = PromptTemplate.from_template("""
Provide insider tips and local advice for {destination} travel.
Preferences: {preferences}, Budget: {budget}.
""")


4. Chains for each Agent

In [ ]:
destination_chain = LLMChain(llm=llm, prompt=destination_prompt, output_key="destination_report")
attractions_chain = LLMChain(llm=llm, prompt=attractions_prompt, output_key="attractions")
itinerary_chain = LLMChain(llm=llm, prompt=itinerary_prompt, output_key="itinerary")
guide_chain = LLMChain(llm=llm, prompt=guide_prompt, output_key="guide")

5. Input from User

In [ ]:
destination = Input("Destination: ")
travel_dates = input("Travel Dates (default June 1, 2023): ") or "June 1, 2023"
duration_days = int(input("Duration (default 3): ") or "3")
preferences = input(Preferences (default: Cultural & Museums, Food & Culinary): ") or "Cultural & Museums, Food & Culinary"
budget = input("Budget (default: moderate): ") or "moderate"

input_variables = {
    "destination": destination,
    "travel_dates": travel_dates,
    "duration_days": duration_days,
    "preferences": preferences,
    "budget": budget
}

6. Sequential Execution with "SequentialChain"

In [ ]:
travel_chain = SequentialChain(
    chains = [destination_chain, attractions_chain, itinerary_chain, guide_chain],
    input_variables=["destination", "travel_dates", "duration_days", "preferences", "budget"],
    output_variables=["destination_report", "attractions", "itinerary", "guide"],
    verbose=True
)

results = travel_chain.run(input_variables)

print("\n--- Travel Plan ---")
print("Destination Overview:\n", results["destination_report"])
print("\n Attractions:\n", results["attractions"])
print("\n Itinerary:\n", results["itinerary"])
print("\n Local Guide:\n", results["guide"])

Supervisor with tools Architecture - A supervisor agent uses a tool-calling agent to decide which tool to use.

In [ ]:
llm = ChatOPENAI(
    model="gpt-4o",
    temperature=0.7,
    openai_api_key=openai_api_key
)

Define web search tool using Tavily

In [ ]:
search_tool = TavilySearchResults(api_key=tavily_api_key, max_results=5)

In [ ]:
destination_researcher = Agent(
    role="Destination Researcher",
    goal="Gather essential information about the destination, travel safety, seasons, and culinary."
    backstory="An expert travel researcher with deep knowledge of global destination and up"
    tools=[search_web],
    llm=llm,
    memory=True,
    verbose=True
)

In [ ]:
Task(
    description=f"""
            Research {destination} for a traveler going on {travel_dates} for {duration_day}
            Include climate, travel safety, currency, transport options, and cultural etiquette 
            """,
    agent=destination_researcher,
    expected_output="Destination research report with key facts and insights."
),

In [ ]:
travel_crew = Crew(
    agents=[destination_researcher, attractions_specialist, itinerary_planner, local_guide],
    tasks=tasks,
    verbose=True,
    manager_agent=manager, # required only for the hierarchical process
    process=Process.hierarchical  # dynamic agent coordination
)

In [ ]:
results = travel_crew.kickoff()